# TP4 — Détection & extraction de points clés biométriques avec Python

Dans ce notebook, nous mettons en œuvre un pipeline complet d'extraction de descripteurs
biométriques pour trois modalités :

- **Visage** : détection de landmarks faciaux et normalisation.
- **Empreintes digitales** : détection de points d'intérêt et normalisation.
- **Voix** : extraction de coefficients MFCC et résumé statistique.

L'objectif est d'analyser ces descripteurs tout en respectant les principes de la
**confiance numérique** (Privacy by Design / By Default, Loi 09-08, RGPD).

Ce notebook est structuré de façon à suivre la grille d'évaluation :

1. Qualité technique du traitement
2. Conception et qualité du notebook
3. Respect de la confiance numérique & CNDP & RGPD
4. Clarté des résultats et interprétation
5. Appréciation globale


In [2]:
# 0. Imports & configuration générale

import os
from pathlib import Path

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import librosa
import librosa.display

import mediapipe as mp  # pour les landmarks faciaux

# Configuration Matplotlib
plt.rcParams["figure.figsize"] = (6, 4)

# Chemins
DATA_DIR = Path("data")
FACES_DIR = DATA_DIR / "faces"
FP_DIR = DATA_DIR / "fingerprints"
VOICES_DIR = DATA_DIR / "voices"

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

print("Dossier faces :", FACES_DIR)
print("Dossier empreintes :", FP_DIR)
print("Dossier voix :", VOICES_DIR)
print("Dossier outputs :", OUTPUT_DIR)

Dossier faces : data\faces
Dossier empreintes : data\fingerprints
Dossier voix : data\voices
Dossier outputs : outputs


## Description des données

Nous supposons l'organisation suivante :

- `data/faces/` : images de visages (`.jpg`, `.png`, ...) de différentes personnes.
- `data/fingerprints/` : images d'empreintes digitales.
- `data/voices/` : fichiers audio (`.wav`) contenant des échantillons de voix.
- `outputs/` : dossier où seront enregistrés les CSV de descripteurs.

Chaque modalité fera l'objet :
- d'une **extraction de descripteurs**,
- d'une **normalisation**,
- d'une **sauvegarde dans un CSV**,
- d'une **visualisation**,
- d'une **analyse critique**.


# 1. Biometrie visage : détection & extraction de landmarks

Dans cette section, nous utilisons **MediaPipe Face Mesh** pour :
- détecter un visage dans l'image,
- extraire les landmarks (points clés),
- normaliser les coordonnées (centrage + division par l'écart-type),
- exporter les descripteurs dans un CSV.

En cas d'échec de détection, l'exemple est marqué `success = False`. 


In [3]:
mp_face = mp.solutions.face_mesh

def extract_face_landmarks(img_bgr: np.ndarray) -> np.ndarray | None:
    """
    Extrait les landmarks faciaux et renvoie un vecteur 1D normalisé (x,y).
    Normalisation : centrage + division par l'écart-type.
    Retourne None en cas d'échec de détection.
    """
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    h, w, _ = img_rgb.shape

    with mp_face.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        result = face_mesh.process(img_rgb)

    if not result.multi_face_landmarks:
        return None

    lm = result.multi_face_landmarks[0]
    points = np.array([[p.x * w, p.y * h] for p in lm.landmark], dtype=np.float32)

    # Normalisation : translation + échelle
    mean = points.mean(axis=0)
    std = points.std(axis=0) + 1e-6
    pts_norm = (points - mean) / std

    return pts_norm.flatten()

In [4]:
face_rows = []

for img_path in FACES_DIR.glob("*"):
    if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
        continue

    img = cv2.imread(str(img_path))
    feats = extract_face_landmarks(img)

    row = {
        "file": img_path.name,
        "success": feats is not None,
    }
    if feats is not None:
        for i, v in enumerate(feats):
            row[f"f{i}"] = float(v)

    face_rows.append(row)

df_faces = pd.DataFrame(face_rows)
csv_faces_path = OUTPUT_DIR / "features_faces.csv"
df_faces.to_csv(csv_faces_path, index=False)

print("Nombre d'images visage traitées :", len(df_faces))
print("Nombre de détections réussies :", df_faces["success"].sum())
df_faces.head()

C:\Users\Asus\PycharmProjects\JupyterProject\Tp4-biometrie-v3\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# Visualisation des landmarks sur un exemple correct

if df_faces["success"].sum() > 0:
    sample_row = df_faces[df_faces["success"]].iloc[0]
    sample_file = sample_row["file"]
    sample_img = cv2.imread(str(FACES_DIR / sample_file))

    # Ici, on recalcul les landmarks pour avoir les points dans l'image d'origine
    img_rgb = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
    h, w, _ = img_rgb.shape

    with mp_face.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        result = face_mesh.process(img_rgb)

    lm = result.multi_face_landmarks[0]
    pts = np.array([[p.x * w, p.y * h] for p in lm.landmark], dtype=np.float32)

    plt.imshow(img_rgb)
    plt.scatter(pts[:, 0], pts[:, 1], s=5)
    plt.title(f"Landmarks faciaux sur {sample_file}")
    plt.axis("off")
    plt.show()
else:
    print("Aucune détection visage réussie pour la visualisation.")

## Analyse des descripteurs faciaux

Les landmarks détectés couvrent la majeure partie du visage : contour, yeux, nez,
bouche, etc. La normalisation par centrage et division par l'écart-type permet
de rendre les descripteurs plus robustes aux translations et à la taille de l'image.

Les cas où la détection échoue (flou, mauvaise pose, occlusion) sont explicitement
marqués `success = False` dans le CSV, ce qui permet de les filtrer lors d'une
éventuelle phase d'apprentissage ou d'évaluation.


# 2. Biometrie empreintes digitales : points d'intérêt & descripteurs

Dans cette section, nous utilisons le détecteur **ORB** (Oriented FAST and Rotated BRIEF)
pour extraire des descripteurs à partir des empreintes digitales :

- Détection de points d'intérêt sur les lignes de crêtes.
- Extraction des descripteurs ORB.
- Mise à dimension fixe et normalisation.
- Sauvegarde dans un CSV + visualisation des points détectés.


In [ ]:
orb = cv2.ORB_create()

def extract_fp_keypoints(img_gray: np.ndarray, max_kp: int = 128) -> np.ndarray | None:
    """
    Extrait des descripteurs ORB et les normalise.
    Renvoie un vecteur 1D de taille fixe (max_kp * dim_descripteur)
    ou None si aucun point n'est détecté.
    """
    kp, des = orb.detectAndCompute(img_gray, None)
    if des is None:
        return None

    # Limiter à max_kp pour dimension fixe
    des = des[:max_kp]
    if des.shape[0] < max_kp:
        pad = np.zeros((max_kp - des.shape[0], des.shape[1]), dtype=des.dtype)
        des = np.vstack([des, pad])

    des = des.astype(np.float32)
    mean = des.mean(axis=0)
    std = des.std(axis=0) + 1e-6
    des_norm = (des - mean) / std

    return des_norm.flatten()

In [ ]:
fp_rows = []

for img_path in FP_DIR.glob("*"):
    if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
        continue

    img_gray = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    feats = extract_fp_keypoints(img_gray)

    row = {
        "file": img_path.name,
        "success": feats is not None,
    }
    if feats is not None:
        for i, v in enumerate(feats):
            row[f"f{i}"] = float(v)

    fp_rows.append(row)

df_fp = pd.DataFrame(fp_rows)
csv_fp_path = OUTPUT_DIR / "features_fingerprints.csv"
df_fp.to_csv(csv_fp_path, index=False)

print("Nombre d'empreintes traitées :", len(df_fp))
print("Nombre de détections réussies :", df_fp["success"].sum())
df_fp.head()

In [ ]:
# Visualisation des points d'intérêt sur une empreinte

if df_fp["success"].sum() > 0:
    sample_fp_row = df_fp[df_fp["success"]].iloc[0]
    sample_fp_file = sample_fp_row["file"]

    img_fp = cv2.imread(str(FP_DIR / sample_fp_file), cv2.IMREAD_GRAYSCALE)
    kp, _ = orb.detectAndCompute(img_fp, None)
    img_kp = cv2.drawKeypoints(img_fp, kp, None)

    plt.imshow(img_kp, cmap="gray")
    plt.title(f"Points d'intérêt ORB sur {sample_fp_file}")
    plt.axis("off")
    plt.show()
else:
    print("Aucune détection ORB réussie pour la visualisation.")

## Analyse des points d'intérêt sur empreintes digitales

Les points d'intérêt ORB se concentrent principalement sur les zones de changement
de texture : bifurcations, terminaisons de crêtes, croisements. Ce comportement
est cohérent avec la notion de minuties utilisée en biométrie.

La normalisation des descripteurs permet de réduire l'influence des variations
de contraste ou d'éclairage. Les cas où aucun point n'est détecté sont marqués
`success = False` et pourront être traités à part (ré-acquisition, exclusion, etc.).


# 3. Biometrie vocale : extraction de MFCC

Dans cette section, nous extrayons des **coefficients cepstraux** (MFCC) à partir
d'enregistrements audio :

- Chargement des fichiers `.wav`.
- Calcul des MFCC.
- Résumé statistique (moyenne et écart-type par coefficient).
- Sauvegarde dans un CSV.
- Visualisation des MFCC sous forme de carte temps / coefficient.


In [ ]:
def extract_mfcc_features(wav_path: Path, n_mfcc: int = 13):
    """
    Calcule les MFCC pour un fichier audio et renvoie :
    - mean : moyenne par coefficient
    - std  : écart-type par coefficient
    - mfcc : matrice brute (n_mfcc x temps)
    - sr   : fréquence d'échantillonnage
    """
    y, sr = librosa.load(wav_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mean = mfcc.mean(axis=1)
    std = mfcc.std(axis=1)
    return mean, std, mfcc, sr

voice_rows = []
mfcc_cache = {}  # pour les visualisations

for wav_path in VOICES_DIR.glob("*.wav"):
    mean, std, mfcc, sr = extract_mfcc_features(wav_path)
    mfcc_cache[wav_path.name] = (mfcc, sr)

    row = {"file": wav_path.name}
    for i, v in enumerate(mean):
        row[f"mfcc{i}_mean"] = float(v)
    for i, v in enumerate(std):
        row[f"mfcc{i}_std"] = float(v)

    voice_rows.append(row)

df_voice = pd.DataFrame(voice_rows)
csv_voice_path = OUTPUT_DIR / "features_voices.csv"
df_voice.to_csv(csv_voice_path, index=False)

print("Nombre de fichiers audio traités :", len(df_voice))
df_voice.head()

In [ ]:
# Visualisation des MFCC pour un échantillon

if len(df_voice) > 0:
    sample_voice = df_voice.iloc[0]["file"]
    mfcc, sr = mfcc_cache[sample_voice]

    plt.figure(figsize=(8, 4))
    librosa.display.specshow(mfcc, x_axis='time', sr=sr)
    plt.colorbar()
    plt.title(f"MFCC pour {sample_voice}")
    plt.tight_layout()
    plt.show()
else:
    print("Aucun fichier audio trouvé pour la visualisation.")

## Analyse des descripteurs vocaux

Les MFCC capturent la structure spectrale de la parole dans une base
plus adaptée que le spectre brut. En calculant la moyenne et l'écart-type
pour chaque coefficient, on obtient un vecteur de taille fixe qui résume
la "signature vocale" de l'échantillon.

Deux échantillons provenant du même locuteur devraient avoir des vecteurs
(mean, std) relativement proches, ce qui permettrait d'utiliser ces descripteurs
pour de l'identification ou de la vérification.


# 4. Confiance numérique, CNDP & RGPD

## Minimisation et finalité

- Seules les données strictement nécessaires au TP sont utilisées :
  images de visages anonymes, empreintes digitales et enregistrements vocaux.
- La finalité est **purement pédagogique** : illustration de techniques
  d'extraction de descripteurs biométriques dans le cadre du module
  *Confiance Numérique et accès biométrique*.

## Transparence et documentation

- La provenance des données est explicitée (données fournies par l'enseignant /
  jeu de données public, à préciser).
- Le pipeline de traitement est entièrement documenté dans ce notebook :
  chargement, détection, extraction, normalisation, export en CSV.

## Sécurité et anonymisation

- Les CSV produits ne contiennent que des vecteurs numériques et des noms
  de fichiers techniques, sans nom, prénom ni identifiant civil.
- Les données sont traitées et stockées **en local** pour le TP, sans
  diffusion sur des services cloud publics.
- Dans un contexte réel, il serait nécessaire :
  - de chiffrer les données biométriques,
  - de limiter les accès (contrôle d'accès, journalisation),
  - de réduire la durée de conservation.

## Réflexion éthique et conformité

- Les données biométriques sont des **données sensibles** au sens de la Loi 09-08
  et du RGPD.
- Leur traitement nécessite un **consentement explicite**, une finalité claire
  et une durée de conservation limitée.
- En cas de fuite, les conséquences peuvent être graves (usurpation d'identité,
  surveillance abusive). Le principe de **Privacy by Design / By Default**
  impose donc de minimiser les données collectées, de sécuriser les systèmes
  par défaut et d'informer clairement les personnes concernées.


In [ ]:
# 5. Clarté des résultats et interprétation

# ----- Statistiques simples pour les visages -----
if df_faces["success"].sum() > 0:
    face_feats = df_faces[df_faces["success"]].filter(like="f").to_numpy()
    face_norms = np.linalg.norm(face_feats, axis=1)

    plt.hist(face_norms, bins=15)
    plt.xlabel("Norme du vecteur de descripteurs visage")
    plt.ylabel("Effectif")
    plt.title("Distribution de la norme des descripteurs faciaux")
    plt.show()
else:
    print("Pas de descripteurs visage valides pour l'histogramme.")

# ----- Statistiques simples pour les voix -----
if len(df_voice) > 0:
    mfcc0_mean = df_voice["mfcc0_mean"]

    plt.hist(mfcc0_mean, bins=10)
    plt.xlabel("MFCC0 (moyenne)")
    plt.ylabel("Effectif")
    plt.title("Distribution de la moyenne du MFCC0")
    plt.show()
else:
    print("Pas de descripteurs vocaux pour l'histogramme.")

## Discussion des résultats, limites et biais

Les histogrammes des normes de descripteurs faciaux montrent que la plupart
des vecteurs se situent dans une plage de valeurs relativement compacte.
Les valeurs extrêmes correspondent à des cas où la détection est moins stable
(images floues, mal cadrées, expression inhabituelle).

Pour les descripteurs vocaux, la distribution de la moyenne du premier MFCC
montre une certaine variabilité entre les échantillons, ce qui est attendu
si les locuteurs ou les phrases prononcées sont différents.

### Limites

- Les jeux de données utilisés sont de taille réduite et ne couvrent pas
  toute la diversité des visages, empreintes et voix.
- La qualité de capture (bruit, éclairage, micro) influence fortement les
  descripteurs.
- Il n'y a pas encore d'évaluation quantitative (taux de faux positifs /
  faux négatifs, EER, etc.).

### Biais possibles

- Biais démographiques si les données proviennent majoritairement d'un même
  groupe (âge, genre, origine, etc.).
- Biais de capture (un seul type de capteur / caméra / micro).
- Biais de sélection (échantillons trop propres par rapport au monde réel).

Ces éléments montrent que la mise en production d'un système biométrique
nécessite des bases de données plus larges, des protocoles de tests rigoureux
et une analyse éthique approfondie.


# 6. Conclusion

Ce notebook met en œuvre un pipeline complet d'extraction de descripteurs
biométriques :

- **Visage** : détection de landmarks faciaux, normalisation et export CSV.
- **Empreintes digitales** : détection de points d'intérêt ORB, normalisation
  des descripteurs et visualisation.
- **Voix** : extraction de MFCC, résumé statistique (moyenne / écart-type),
  visualisation spectrale.

L'ensemble du travail est documenté, accompagné de visualisations et d'une
réflexion sur la confiance numérique (Loi 09-08, RGPD, Privacy by Design).

Ce type de pipeline pourrait servir de base à un système d'authentification
ou d'identification biométrique, à condition d'ajouter :
- des modules de comparaison (distances, scores),
- des métriques de performance (FAR, FRR, EER),
- des mécanismes de protection avancée des données sensibles.

Dans le cadre de ce TP, l'objectif pédagogique d'extraction et d'analyse de
descripteurs biométriques est atteint.
